# Simple PDF Form Analyzer

The following sections will demonstrate how to:
1. load a standard corporate PDF form (e.g., onboarding, invoice, feedback form).
2. extract specific information using PyPDF2.
3. save the data to a specific location


We want to show you how to easily **implement a pipeline** that takes on mundane chores for you, and how to do this in a nice, **user-friendly interface**.


## Loading a PDF form

First off, we need to somehow read the PDF into Python. To do that, we'll use [PyPDF2](https://pypdf2.readthedocs.io/en/3.0.0/modules/PdfReader.html#PyPDF2.PdfReader.get_fields).

Here's how that looks like:

In [35]:
from PyPDF2 import PdfReader

pdf_path = "data/report_lovegood.pdf"   # we set the file path
reader = PdfReader(pdf_path)            # initialize a PyPDF2 PdfReader class instance 

fields = reader.get_fields()            # extract field data if this PDF contains interactive form fields
data = {key: value.get('/V', '') for key, value in fields.items()}

# --------------------
# some info:
# field is a dict with several keys; key "/V" stores the value (what the user entered),
# key "/T" stores the field's title, "FT" what field type, etc. We only want the value.
# -----------------------------------------------


# let's take a look how the data dict looks like:
print(data)    

{'Reporting Month': 'August 2025', 'Corporation Name': 'Important Company Association Co', 'Registered Agent': 'Luna Lovegood', 'Registered Agent ID': 'HRM20-1021', 'Registered Department': 'Human Ressource Management', 'City ZIP County': 'Important Village, 93915, Nonimportance County', 'Job Title': 'Legal Counselor ', 'NameRow1': 'Project AA', 'DescriptionRow1': 'Important Management Stuff', 'Starting DateRow1': '01.04.2021', 'Ending DateRow1': '31.03.2031', 'NameRow2': 'Project CA', 'DescriptionRow2': 'Important Legal Stuff', 'Starting DateRow2': '01.04.2018', 'Ending DateRow2': '01.10.2021', 'NameRow3': 'Project DA', 'DescriptionRow3': 'Unimportant Management Stuff', 'Starting DateRow3': '01.02.2016', 'Ending DateRow3': '01.04.2018', 'NameRow4': 'Project AD', 'DescriptionRow4': 'Legal and Management Stuff', 'Starting DateRow4': '01.04.2018', 'Ending DateRow4': '01.12.2025', 'NameRow5': '', 'DescriptionRow5': '', 'Starting DateRow5': '', 'Ending DateRow5': '', 'NameRow6': '', 'Descr

## Clean the data

Now we extracted the text in a *somewhat* structured form, but we've got several issues:

- some fields are empty; information is useless to us
- the 'Yes'/'No' answers have a weird formatting
- some information is related; NameRow, DescriptionRow, etc. all refer to a table and cannot be seen independently.

As a next step, we thus want to clean the data. Here's how we do this.

In [36]:
import pandas as pd
import numpy as np

# 1. remove empty fields
data = pd.DataFrame([data])                 # load data into a pandas dataframe
data = data.loc[:, ~(data == '').all()]     # remove columns with empty entries
data.head()

# 2. reformat the Yes/No outputs
for key, value in data.items():
    val = value[0]  # the actual cell value
    choices = ["Yes", "No"]
    one_word = len(str(val).split()) < 2
    mask = [name in str(val) for name in choices if one_word]
    if any(mask):
        choice = choices[np.where(mask)[0][0]]
    else:
        choice = val
    data.at[0, key] = choice
data.head()

# 3. separate information into metadata and tabular data
metadata = {}
row_data = {}

for key, value in data.items():
    
    if "Row" in key:    # if "Row" is part of the key name, we know it must be tabular data
        
        # go through all the possible prefixes we had
        for prefix in ["NameRow", "DescriptionRow", "Starting DateRow", "Ending DateRow"]:

            if key.startswith(prefix):

                # remove the 'Row' from the prefix -> line number remains
                field_name = prefix.replace("Row", "")

                # remove the "NameRow", "DescriptionRow", etc. from the key name -> only the row number remains
                row_idx = key.replace(prefix, "")
                if row_idx.isdigit():
                    row_idx = int(row_idx)
                    if row_idx not in row_data:
                        row_data[row_idx] = {}
                    row_data[row_idx][field_name] = value[0]
                break
    # if not, it must be regular metadata
    else:
        metadata[key] = value[0]

# Convert row_data into DataFrame
tabulardata_df = pd.DataFrame.from_dict(row_data, orient="index").dropna(how="all")
metadata_df = pd.DataFrame.from_dict(metadata, orient="index").dropna(how="all")

In [37]:
tabulardata_df.head()

,Name,Description,Starting Date,Ending Date
1,Project AA,Important Management Stuff,01.04.2021,31.03.2031
2,Project CA,Important Legal Stuff,01.04.2018,01.10.2021
3,Project DA,Unimportant Management Stuff,01.02.2016,01.04.2018
4,Project AD,Legal and Management Stuff,01.04.2018,01.12.2025


In [39]:
metadata_df.head(20)

,0
Reporting Month,August 2025
Corporation Name,Important Company Association Co
Registered Agent,Luna Lovegood
Registered Agent ID,HRM20-1021
Registered Department,Human Ressource Management
City ZIP County,"Important Village, 93915, Nonimportance County"
Job Title,Legal Counselor
Brief statement of type of business of the corporation,Important Management and Legal Stuff
Have there been any business trips,No
If yes provide destination and duration,-


## Save the data

Lastly, we want to save the data to a results folder, as a `csv`-file.
Here's how we do this.

In [ ]:
import os

save_path = "data/results"
file_name = "example"

# create a new directory if it doesn't exist already
os.makedirs(save_path, exist_ok=True)


# save metadata to csv-file
metadata_path = os.path.join(save_path, f"{file_name}_metadata.csv")
metadata_df.to_csv(metadata_path, index=True)
print("Metadata saved to:", metadata_path)

# save tabular data to csv-file
tabulardata_path = os.path.join(save_path, f"{file_name}_tabulardata.csv")
tabulardata_df.to_csv(tabulardata_path, index=True)
print("Tabular data saved to:", tabulardata_path)


KeyError: 0